In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset

In [ ]:
checkpoint = 'AlanRobotics/ruT5-base'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
raw_dataset = load_dataset('sberquad')
raw_dataset['train'][0]

In [8]:
max_length = 512
def preprocess_dataset(example):
    tokenized_answers = tokenizer(example['answers']['text'][0])
    tokenized_context = tokenizer(example['context'], example['question'], max_length=max_length, truncation=True)
    tokenized_context['labels'] = tokenized_answers['input_ids']
    return tokenized_context

In [ ]:
tokenized_dataset = raw_dataset.map(preprocess_dataset)
tokenized_dataset = tokenized_dataset.remove_columns(['id', 'title', 'context', 'question', 'answers'])
tokenized_dataset.set_format('torch')

In [20]:
batch_size = 4
num_train_epochs = 8
logging_steps = len(tokenized_dataset["train"]) // batch_size
model_name = checkpoint.split('/')[-1]


args = Seq2SeqTrainingArguments(
    output_dir=f'{model_name}-finetuned',
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_steps=logging_steps
    )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [21]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [22]:
trainer = Seq2SeqTrainer(
    model,
    args,
    data_collator,
    tokenized_dataset['train'],
    tokenized_dataset['validation'],
    tokenizer
)

In [23]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [25]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
trainer.train()

In [ ]:
context = 'И́лон Рив Маск род. 28 июня 1971[1][2][…], Претория, ЮАР) — американский предприниматель, инженер[5] и миллиардер. Основатель, генеральный директор и главный инженер компании SpaceX; инвестор, генеральный директор и архитектор продукта компании Tesla; основатель The Boring Company; соучредитель Neuralink и OpenAI; владелец Twitter. 7 января 2021 года, с состоянием по оценочным данным в 185 млрд $, впервые стал богатейшим человеком планеты, сместив на второе место основателя Amazon Джеффа Безоса[6]. 1 ноября 2021 года стал первым человеком в истории, чьё состояние достигло отметки в 300 млрд $[4][⇨]. Маск родился и вырос в Претории, ЮАР. Некоторое время учился в Преторийском университете, а в 17 лет переехал в Канаду. Поступил в Университет Куинс в Кингстоне и через два года перевёлся в Пенсильванский университет, где получил степень бакалавра по экономике и физике. В 1995 году переехал в Калифорнию, чтобы учиться в Стэнфордском университете, но вместо этого решил заняться бизнесом и вместе со своим братом Кимбалом  (англ.)рус. стал соучредителем компании Zip2, занимавшейся разработкой программного обеспечения для интернета. В 1999 году компания была приобретена Compaq за 307 миллионов долларов. В том же году Маск стал соучредителем онлайн-банка X.com, который в 2000 году конгломеративным путем консолидировался с Confinity и образовал PayPal. В 2002 году компания была куплена eBay за 1,5 миллиарда долларов. '
question = 'Какую компанию возглаваляет?'
tokenized_sentence = tokenizer(context, question, return_tensors='pt').to('cuda')
res = model.generate(**tokenized_sentence)

In [47]:
tokenizer.decode(res[0])

'<pad> SpaceX</s>'

In [50]:
model.push_to_hub('AlanRobotics/ruT5_q_a')

Configuration saved in /tmp/tmph6l6vuge/config.json
Model weights saved in /tmp/tmph6l6vuge/pytorch_model.bin
Uploading the following files to AlanRobotics/ruT5_q_a: config.json,pytorch_model.bin


CommitInfo(commit_url='https://huggingface.co/AlanRobotics/ruT5_q_a/commit/2f02eefbbdadec157754b19d71ca15da3aa115dc', commit_message='Upload T5ForConditionalGeneration', commit_description='', oid='2f02eefbbdadec157754b19d71ca15da3aa115dc', pr_url=None, pr_revision=None, pr_num=None)